So this is yet another RoBERTa tweet sentiment extraction notebook. However with some different things and approaches used so I hope you like it. And if so then please upvote it ;-)

My original work in this competition was inspired by the kernels of Chris Deotte. 

This kernel is based somewhat on that work yet offers a couple of different/new attempts:
* It is using the TFRobertaForQuestionAnswering model that was recently released by Huggingface. And with custom head layers added.
* It is using the default RobertaTokenizer
* It is using some different preprocessing where I use more of the default tokenizer capabilities.
* Label Smoothing.
* Added some simple post-processing.

Update: In the last version I've added support for running on TPU. Note that this only works for Training. Inference and submission should be done on GPU as TPU is not allowed in this competition.

In [1]:
# Import Modules
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
# Show versions of Tensorflow
print('TF version',tf.__version__)

TF version 2.2.0


In [3]:
# Set other Seeds
SEED = 4262
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
# Set strategy for tpu
USE_TPU = False
if USE_TPU:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.OneDeviceStrategy(device = "/gpu:0")

In [5]:
# Constants
MAX_LEN = 128
FOLDS = 5
EPOCHS = 3
VERBOSE = 1
ROBERTA_BASE_PATH = '../input/tf-roberta/'

BATCH_SIZE = 32 * strategy.num_replicas_in_sync
print('BatchSize: {}'.format(BATCH_SIZE))

LR = 3e-5 * strategy.num_replicas_in_sync
print('LearningRate: {}'.format(LR))

# Set the following to True after training to make a submission
INFERENCE = True

BatchSize: 32
LearningRate: 3e-05


In [6]:
# Read Train Data (I read in a multiple of batch size 256 when training on TPU)
#train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv', nrows = 26880).fillna('')
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv').fillna('')

# Summary
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [7]:
# Read Test Data
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')

# Summary
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


For the tokenizer I'am using the default Huggingface Roberta Tokenizer and loading the vocab and merges file.

In [8]:
# Tokenizer
tokenizer = RobertaTokenizer(vocab_file = ROBERTA_BASE_PATH + 'vocab-roberta-base.json',
                             merges_file = ROBERTA_BASE_PATH + 'merges-roberta-base.txt',
                             add_prefix_space = True,
                             do_lower_case = True)

# Training Data

For processing of the training data I mostly use the proces as was shown in one of the earlier kernels from Abhishek Thakur. Main difference is I'am using the default Roberta Tokenizer. Also I just use the tokenizers functionality to generate the complete input sample.

In [9]:
# Pre Process Training Data
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN), dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN), dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN), dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN), dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN), dtype='int32')

for k in range(ct):
    # Process Text
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    
    # Skip rows where Text1 is empty (RobertaTokenizer crashes: https://github.com/huggingface/transformers/issues/3809)
    if text1 != '':
        # Encode Full input sample
        input_encoded = tokenizer.encode_plus(text1, train.loc[k,'sentiment'], add_special_tokens = True, max_length = MAX_LEN)
        input_ids_sample = input_encoded["input_ids"]
        attention_mask_sample = input_encoded["attention_mask"]

        # Attention Mask
        attention_mask[k,:len(attention_mask_sample)] = attention_mask_sample

        # Input Ids
        input_ids[k,:len(input_ids_sample)] = input_ids_sample

        # Find overlap between Full Text and Selected Text
        idx = text1.find(text2)
        chars = np.zeros((len(text1)))
        chars[idx:idx + len(text2)] = 1
        k_ids = tokenizer.encode(text1, add_special_tokens = False) 

        # ID_OFFSETS
        offsets = [] 
        idx = 0
        for t in k_ids:
            w = tokenizer.decode([t])
            offsets.append((idx,idx+len(w)))
            idx += len(w)

        # Get Start and End Tokens
        toks = []
        for i,(a,b) in enumerate(offsets):
            sm = np.sum(chars[a:b])
            if sm>0: 
                toks.append(i) 
        if len(toks) > 0:
            start_tokens[k,toks[0]+1] = 1
            end_tokens[k,toks[-1]+1] = 1

# Test Data

In [10]:
# Pre Process Test Data
ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(ct):
    # Process Text
    text1 = " "+" ".join(test.loc[k,'text'].split())
    
    # Skip rows where Text1 is empty (RobertaTokenizer crashes: https://github.com/huggingface/transformers/issues/3809)
    if text1 != '':
        # Encode Full input sample
        input_encoded = tokenizer.encode_plus(text1, test.loc[k, 'sentiment'], add_special_tokens = True, max_length = MAX_LEN)
        input_ids_sample = input_encoded["input_ids"]
        attention_mask_sample = input_encoded["attention_mask"]

        # Attention Mask
        attention_mask_t[k,:len(attention_mask_sample)] = attention_mask_sample

        # Input Ids
        input_ids_t[k,:len(input_ids_sample)] = input_ids_sample

## Metric

In [11]:
# Metric
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    
    if (len(a)==0) & (len(b)==0): 
        return 0.5
    
    c = a.intersection(b)
    
    return float(len(c)) / (len(a) + len(b) - len(c))

## Models

In [12]:
def custom_loss(y_true, y_pred):
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits = False, label_smoothing = 0.1)
    loss = tf.reduce_mean(loss)
    
    return loss

When setting the config I increase the dropout for the attention layers a bit.

In [13]:
# Config
config = RobertaConfig.from_pretrained(ROBERTA_BASE_PATH+'config-roberta-base.json')
#config.attention_probs_dropout_prob = 0.15
print(config)

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "eos_token_id": 2,
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "type_vocab_size": 1,
  "vocab_size": 50265
}



When building the model I use the recently added TFRobertaForQuestionAnswering model as a basis.

In [14]:
def build_model():
    # Create Model
    with strategy.scope():      
        ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
        att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
        tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

        roberta_model = TFRobertaForQuestionAnswering.from_pretrained(ROBERTA_BASE_PATH + 'pretrained-roberta-base.h5', config = config)
        x = roberta_model(ids, attention_mask = att, token_type_ids = tok)

        x1 = tf.keras.layers.Dropout(0.30)(x[0]) 
        x1 = tf.keras.layers.Activation('softmax')(x1)

        x2 = tf.keras.layers.Dropout(0.30)(x[1]) 
        x2 = tf.keras.layers.Activation('softmax')(x2)

        model = tf.keras.models.Model(inputs = [ids, att, tok], outputs=[x1, x2])
        optimizer = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(loss = custom_loss, optimizer = optimizer)

        return model

## Train Model

After training the model I apply some simple post processing. See for yourself if you want to use it. In the situation where the start position is after the end position I try to find if the 2nd or 3rd options provides a solution for that situation.

With the similarity between 'text' and 'selected_text' when the sentiment is neutral I just use the complete 'text'. Risky because we don't know the data for the private board..but worth a try ;-)

In [15]:
# Placeholders
jac, jac1 = [], []
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    # Clear session and create Model
    K.clear_session()
    model = build_model()
        
    # Callbacks
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint('roberta-%i.h5'%(fold), monitor = 'val_loss', verbose = 1, save_best_only = True,
                                                          save_weights_only = True, mode = 'auto', save_freq = 'epoch')
    
    if not INFERENCE:
        # Train Model 
        model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
                  epochs = EPOCHS, 
                  batch_size = BATCH_SIZE, 
                  verbose = VERBOSE, 
                  callbacks = [model_checkpoint],
                  validation_data = ([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], [start_tokens[idxV,], end_tokens[idxV,]]),
                  shuffle = True)

        print('Loading model...')
        model.load_weights(f'roberta-{fold}.h5')        
    else:
        print('Loading model...')
        model.load_weights(f'/kaggle/input/tensorflow-roberta-qa-model/roberta-{fold}.h5')
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose = VERBOSE)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t, attention_mask_t, token_type_ids_t], verbose = VERBOSE)
    preds_start += preds[0] / skf.n_splits
    preds_end += preds[1] / skf.n_splits
    
    # Display Fold Jaccard
    all = []
    all1 = []

    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        
        # Encode Text
        text1 = " "+" ".join(train.loc[k,'text'].split())
        enc = tokenizer.encode(text1, add_special_tokens = False)
        
        if train.loc[k,'sentiment'] == 'neutral':
            st = " ".join(train.loc[k,'text'].split())
            st1 = " ".join(train.loc[k,'text'].split())
        else:
            # Check if start comes after end
            if a > b:
                st = text1
                st1 = text1

                # Sort according to max probabilities and get the indices            
                start_sort = np.argsort(oof_start[k,])[::-1] 
                end_sort = np.argsort(oof_end[k,])[::-1]

                a1 = start_sort[1]
                b1 = end_sort[1]
                a2 = start_sort[2]
                b2 = end_sort[2]

                # Try the next 2 positions..if one of them has the correct order
                if a1 < b1:
                    st1 = tokenizer.decode(enc[a1-1:b1])
                elif a2 < b2:
                    st1 = tokenizer.decode(enc[a2-1:b2])   
            else:
                st = tokenizer.decode(enc[a-1:b])
                st1 = tokenizer.decode(enc[a-1:b])

        # Store Results
        all.append(jaccard(st, train.loc[k,'selected_text']))
        all1.append(jaccard(st1, train.loc[k,'selected_text']))

    jac.append(np.mean(all))
    jac1.append(np.mean(all1))
    
    print('>>>> FOLD %i Jaccard ='%(fold+1), np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1), np.mean(all1))
    
    print()

#########################
### FOLD 1
#########################
Loading model...
Predicting OOF...
172/172 [==============================] - 28s 163ms/step
Predicting Test...
111/111 [==============================] - 18s 165ms/step
>>>> FOLD 1 Jaccard = 0.7106094664162447
>>>> FOLD 1 Jaccard = 0.7106704071002036

#########################
### FOLD 2
#########################
Loading model...
Predicting OOF...
172/172 [==============================] - 28s 163ms/step
Predicting Test...
111/111 [==============================] - 18s 165ms/step
>>>> FOLD 2 Jaccard = 0.7007708686451285
>>>> FOLD 2 Jaccard = 0.7007656922696233

#########################
### FOLD 3
#########################
Loading model...
Predicting OOF...
172/172 [==============================] - 28s 164ms/step
Predicting Test...
111/111 [==============================] - 18s 166ms/step
>>>> FOLD 3 Jaccard = 0.6990822640943382
>>>> FOLD 3 Jaccard = 0.6991504416621909

#########################
### FOLD 4
###############

In [16]:
print(f'=== OVERALL 5 Fold CV Jaccard                  = {np.mean(jac)}')
print(f'=== OVERALL 5 Fold CV Jaccard - Post Processed = {np.mean(jac1)}')

=== OVERALL 5 Fold CV Jaccard                  = 0.7040920055422928
=== OVERALL 5 Fold CV Jaccard - Post Processed = 0.7040574448277765


# Kaggle Submission

In [17]:
# Generate final results for submission
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    
    # Encode Text
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1, add_special_tokens = False)
    
    if test.loc[k, 'sentiment'] == 'neutral':
        st = " ".join(test.loc[k,'text'].split())
    else:
        # Check if start comes after end
        if a > b:  
            st = text1

            # Sort according to max probabilities and get the indices            
            start_sort = np.argsort(preds_start[k,])[::-1] 
            end_sort = np.argsort(preds_end[k,])[::-1]

            a1 = start_sort[1]
            b1 = end_sort[1]
            a2 = start_sort[2]
            b2 = end_sort[2]

            # Try the next 2 positions..if one of them has the correct order
            if a1 < b1:
                st = tokenizer.decode(enc[a1-1:b1])
            elif a2 < b2:
                st = tokenizer.decode(enc[a2-1:b2])  
        else:
            st = tokenizer.decode(enc[a-1:b])

    all.append(st)

In [18]:
# Store Submission and show some results
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
3320,6d68218d8b,tMeme> My besties. If only was there. on Twitpic: http...,neutral,tMeme> My besties. If only was there. on Twitpic: http:/...
2412,e8641ec31d,Phase 2 was a success. Self discovery is vital. Phase 3 ...,positive,success.
2622,15db00c707,OmG U NasTY,negative,OmG U NasTY
1369,2b8b0a7d10,Is twubbing again,neutral,Is twubbing again
125,410dd99aa3,man im so sad school is ending but then again high scho...,neutral,man im so sad school is ending but then again high schoo...
347,b2e3ebc647,"waiting for aaron to get into town, and then leave him f...",negative,booo
2787,0aec5ff9a0,is *ugh* what a miserable looking day. 54 degrees. Where...,negative,miserable
401,97e53162ff,dear oh dear.....,negative,dear oh dear.....
3380,fa625455c7,Everyone is working tonight! I`m bored,negative,bored
1053,196e5dec12,They wouldn`t reverse any of my overdraft fees.,negative,They wouldn`t reverse any of my overdraft fees.
